In [1]:
import osgeo.ogr

In [2]:
import psycopg2

In [3]:
import numpy as np

In [4]:
import shapely

In [5]:
import shapely.wkt

In [6]:
import shapely.ops

In [7]:
import pyproj

In [8]:
import pandas as pd
from pandas import DataFrame, Series

def getDataFrame(databaseName, query):
    connection = psycopg2.connect(database=databaseName)
    cursor = connection.cursor()
    cursor.execute(query)
    colnames = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    dataframe = DataFrame(rows, columns=colnames)
    cursor.close()
    connection.close()
    
    return dataframe

In [9]:
src_proj = pyproj.Proj(proj="longlat", ellps="WGS84", datum="WGS84")

In [10]:
dst_proj = pyproj.Proj(proj="moll", lon_0=0, x_0=0, y_0=0, ellps="WGS84", datum="WGS84", units="m")

In [11]:
def latlong_to_mollweide(longitude, latitude, depth=0):
    return pyproj.transform(src_proj, dst_proj, longitude, latitude)

In [12]:
def transformWkt(wkt):
    geometry = shapely.wkt.loads(wkt)
    #eq_point = shapely.geometry.Point(eq_geometry.x, eq_geometry.y)        
    return shapely.ops.transform(latlong_to_mollweide, geometry)    

In [13]:
def getNearestFaults(earthquake, faults):
    #transform the earthquake point geometry to the mollewide project and then buffer it by 20km
    eq_transformed = transformWkt(earthquake.geometry.values[0]).buffer(10000)
    
    tgeo = faults.geometry.apply(lambda x: transformWkt(x))
    tgeo_index = tgeo.apply(lambda x:x.intersects(eq_transformed))
    
    return faults[tgeo_index]

In [14]:
eq = getDataFrame("earth-monitor", "SELECT id, place, ST_AsText(geometry) as geometry FROM earthquakes;")

In [15]:
ft = getDataFrame("earth-monitor", "SELECT id, faultid, name, ST_AsText(geometry) as geometry FROM faults;")

Get a list of events that have "Hayward" in the place field.  This will allow us to gut check the results.

In [16]:
eq[eq.place.apply(lambda x: x.find("Hayward") > 0)]

,id,place,geometry
6943,14162,"2km ESE of Hayward, California",POINT Z (-122.0578 37.6572 4.7)
29510,36692,"2km SE of Hayward, California",POINT Z (-122.0653 37.6537 5)
29562,36742,"1km WSW of Hayward, California",POINT Z (-122.0973 37.6643 4.5)
33588,40736,"2km SSE of Hayward, California",POINT Z (-122.068 37.6513 5.5)
62578,69714,"4km SE of Hayward, California",POINT Z (-122.0483 37.6385 1.5)
67850,74993,"1km SE of Hayward, California",POINT Z (-122.0648333 37.6595 7.12)
69648,76792,"0km SE of Hayward, California",POINT Z (-122.0742 37.6635 2.6)
70420,77564,"2km SE of Hayward, California",POINT Z (-122.0617 37.6572 4.5)
90349,97525,"3km SE of Hayward, California",POINT Z (-122.0548 37.6435 0.5)
102167,109358,"4km SE of Hayward, California",POINT Z (-122.0503 37.6387 1.3)


Select one of the events from the list of Hayward events above.

In [17]:
test_eq = eq[eq.id == 36692]

In [18]:
test_eq

,id,place,geometry
29510,36692,"2km SE of Hayward, California",POINT Z (-122.0653 37.6537 5)


In [19]:
nearest_faults = getNearestFaults(test_eq, ft)

In [20]:
nearest_faults

,id,faultid,name,geometry
1007,997,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.108140085396 37.69764849...
1047,1028,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.125805699473 37.71016507...
1049,1029,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.126075711025 37.71071005...
1504,1469,380,Chabot fault,"LINESTRING(-122.005410161149 37.607187072926,-..."
1509,1468,380,Chabot fault,MULTILINESTRING((-121.994584911064 37.58765584...
1510,1470,380,Chabot fault,MULTILINESTRING((-121.974258223737 37.58942059...
1512,1472,380,Chabot fault,MULTILINESTRING((-121.970903150859 37.57839101...
1980,1906,55,"Hayward fault zone, northern Hayward section (...",MULTILINESTRING((-122.124224702108 37.72369357...
2204,2103,55,"Hayward fault zone, southern Hayward section (...","LINESTRING(-122.123428667199 37.7216986492741,..."
2266,2163,55,"Hayward fault zone, northern Hayward section (...",MULTILINESTRING((-122.135347140358 37.73770610...


In [21]:
test_eq2 = eq[eq.id == 177492]

In [22]:
test_eq2

,id,place,geometry
170276,177492,"3km SE of Hayward, California",POINT Z (-122.0505 37.6458333 5.39)


In [23]:
nearest_faults2 = getNearestFaults(test_eq2, ft)

In [24]:
nearest_faults2

,id,faultid,name,geometry
1007,997,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.108140085396 37.69764849...
1047,1028,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.125805699473 37.71016507...
1049,1029,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.126075711025 37.71071005...
1504,1469,380,Chabot fault,"LINESTRING(-122.005410161149 37.607187072926,-..."
1509,1468,380,Chabot fault,MULTILINESTRING((-121.994584911064 37.58765584...
1510,1470,380,Chabot fault,MULTILINESTRING((-121.974258223737 37.58942059...
1512,1472,380,Chabot fault,MULTILINESTRING((-121.970903150859 37.57839101...
1980,1906,55,"Hayward fault zone, northern Hayward section (...",MULTILINESTRING((-122.124224702108 37.72369357...
2204,2103,55,"Hayward fault zone, southern Hayward section (...","LINESTRING(-122.123428667199 37.7216986492741,..."
2269,2164,55,"Hayward fault zone, southern Hayward section (...",MULTILINESTRING((-122.018874505054 37.60895697...


In [26]:
eq[eq.place.apply(lambda x: x.find("San Ramon") > 0)]

,id,place,geometry
8890,16113,"4km WNW of San Ramon, California",POINT Z (-122.0312 37.7898 5.5)
16208,23420,"0km SSW of San Ramon, California",POINT Z (-121.9788 37.778 13.7)
16909,24129,"6km ESE of San Ramon, California",POINT Z (-121.9043 37.7592 9.5)
20753,27971,"2km N of San Ramon, California",POINT Z (-121.9782 37.8005 19.3)
22455,29677,"1km NE of San Ramon, California",POINT Z (-121.9652 37.7917 18.6)
27038,34245,"2km NE of San Ramon, California",POINT Z (-121.9603 37.8002 19.3)
27040,34249,"3km NNE of San Ramon, California",POINT Z (-121.9588 37.803 18.9)
32897,40054,"3km ESE of San Ramon, California",POINT Z (-121.9383 37.768 9.9)
44249,51453,"1km SSW of San Ramon, California",POINT Z (-121.9848 37.7708 5.7)
46393,53597,"3km SSW of San Ramon, California",POINT Z (-121.9938 37.7492 7.3)


In [27]:
test_eq3 = eq[eq.id == 323812]

In [28]:
test_eq3

,id,place,geometry
316588,323812,"1km ESE of San Ramon, California",POINT Z (-121.9666667 37.7746667 8.4)


In [29]:
nearest_faults3 = getNearestFaults(test_eq3, ft)

In [30]:
nearest_faults3

,id,faultid,name,geometry
1351,1315,54,"Calaveras fault zone, northern Calaveras secti...","LINESTRING(-121.951606028095 37.7323957738468,..."
1360,1328,54,"Calaveras fault zone, northern Calaveras secti...",MULTILINESTRING((-121.834748599867 37.50984902...
1370,1335,54,"Calaveras fault zone, northern Calaveras secti...",MULTILINESTRING((-121.831640469132 37.49795936...
1376,1342,54,"Calaveras fault zone, northern Calaveras secti...","LINESTRING(-121.999820630917 37.7997507502978,..."
1397,1361,54,"Calaveras fault zone, northern Calaveras secti...",MULTILINESTRING((-121.831639471784 37.49808536...
2007,1930,480,Franklin fault,MULTILINESTRING((-122.042686226288 37.84936159...
2108,2026,54,"Calaveras fault zone, northern Calaveras secti...",MULTILINESTRING((-121.982796045501 37.78154123...
2951,2824,399,Mocho fault,"LINESTRING(-121.82040904564 37.7048045392989,-..."
2985,2858,376,Moraga fault,MULTILINESTRING((-122.097016889018 37.77934770...
3165,3030,54,"Calaveras fault zone, northern Calaveras secti...",MULTILINESTRING((-121.982490037684 37.78199722...
